##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Оценщики

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/estimator"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ru/guide/estimator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/guide/estimator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/ru/guide/estimator.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

Этот документ знакомит с `tf.estimator` — высокоуровневым TensorFlow
API. Оценщики включают следующие действия:

* обучение
* оценка
* предсказание
* экспорт для serving

Вы можете использовать сделанные нами оценщики или
собственные кастомные оценщики.  Все оценщики предварительно сделанные или кастомные являются
классами основанными на классе `tf.estimator.Estimator`.

Для быстрого примера см. [Учебники оценщиков](../../tutorials/estimator/linear.ipynb). Для обзора дизайна API, см. [статью](https://arxiv.org/abs/1708.02637).

## Преимущества

Аналогично `tf.keras.Model`, `estimator` это абстракция на уровне модели. `tf.estimator` предоставляет некоторые возможности, которые все еще находятся в стадии разработки для `tf.keras`. Это:

  * Обучение на сервере параметров
  * Полная [TFX](http://tensorflow.org/tfx) интеграция.

## Возможности оценщиков
Оценщики предоставляют следующие преимущества:

* Вы можете запускать модели на основе оценщиков на локальном компьютере или в распределенной многосерверной среде без изменения вашей модели. Кроме того, вы можете запускать модели на основе оценщиков на CPU, GPU, или TPU без изменения исходного кода вашей модели.
* Оценщики обеспечивают безопасный распределенный цикл обучения, который контролирует как и когда:    
    * загружать данные
    * обрабатывать исключения
    * создавать файлы чекпоинтов и восстанавливаться после сбоев
    * сохранять сводные данные для TensorBoard

При написании приложения с оценщиками, вы должны отделить конвейер входных данных
от модели. Это разделение упрощает эксперименты с
различными наборами данных.

## Готовые оценщики

Pre-made Estimators enable you to work at a much higher conceptual level than the base TensorFlow APIs. You no longer have to worry about creating the computational graph or sessions since Estimators handle all the "plumbing" for you. Furthermore, pre-made Estimators let you experiment with different model architectures by making only minimal code changes.  `tf.estimator.DNNClassifier`, for example, is a pre-made Estimator class that trains classification models based on dense, feed-forward neural networks.

### Structure of a pre-made Estimators program

A TensorFlow program relying on a pre-made Estimator typically consists of the following four steps:

#### 1. Write one or more dataset importing functions.

For example, you might create one function to import the training set and another function to import the test set. Each dataset importing function must return two objects:

* a dictionary in which the keys are feature names and the values are Tensors (or SparseTensors) containing the corresponding feature data
* a Tensor containing one or more labels

For example, the following code illustrates the basic skeleton for an input function:

```
def input_fn(dataset):
    ...  # manipulate dataset, extracting the feature dict and the label
    return feature_dict, label
```

See [data guide](../../guide/data.md) for details.

#### 2. Define the feature columns.

Each `tf.feature_column` identifies a feature name, its type, and any input pre-processing. For example, the following snippet creates three feature columns that hold integer or floating-point data. The first two feature columns simply identify the feature's name and type. The third feature column also specifies a lambda the program will invoke to scale the raw data:

```
# Define three numeric feature columns.
population = tf.feature_column.numeric_column('population')
crime_rate = tf.feature_column.numeric_column('crime_rate')
median_education = tf.feature_column.numeric_column(
  'median_education',
  normalizer_fn=lambda x: x - global_education_mean)
```
For further information, see the [feature columns tutorial](https://www.tensorflow.org/tutorials/keras/feature_columns).

#### 3. Instantiate the relevant pre-made Estimator.

For example, here's a sample instantiation of a pre-made Estimator named `LinearClassifier`:

```
# Instantiate an estimator, passing the feature columns.
estimator = tf.estimator.LinearClassifier(
  feature_columns=[population, crime_rate, median_education])
```
For further information, see the [linear classifier tutorial](https://www.tensorflow.org/tutorials/estimator/linear).

#### 4. Call a training, evaluation, or inference method.

For example, all Estimators provide a `train` method, which trains a model.

```
# `input_fn` is the function created in Step 1
estimator.train(input_fn=my_training_set, steps=2000)
```
You can see an example of this below.

### Benefits of pre-made Estimators

Pre-made Estimators encode best practices, providing the following benefits:

* Best practices for determining where different parts of the computational graph should run, implementing strategies on a single machine or on a
    cluster.
*   Best practices for event (summary) writing and universally useful
    summaries.

If you don't use pre-made Estimators, you must implement the preceding features yourself.

## Custom Estimators

The heart of every Estimator—whether pre-made or custom—is its *model function*, which is a method that builds graphs for training, evaluation, and prediction. When you are using a pre-made Estimator, someone else has already implemented the model function. When relying on a custom Estimator, you must write the model function yourself.

## Recommended workflow

1. Assuming a suitable pre-made Estimator exists, use it to build your first model and use its results to establish a baseline.
2. Build and test your overall pipeline, including the integrity and reliability of your data with this pre-made Estimator.
3. If suitable alternative pre-made Estimators are available, run experiments to determine which pre-made Estimator produces the best results.
4. Possibly, further improve your model by building your own custom Estimator.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

In [0]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

## Create an Estimator from a Keras model

You can convert existing Keras models to Estimators with `tf.keras.estimator.model_to_estimator`. Doing so enables your Keras
model to access Estimator's strengths, such as distributed training.

Instantiate a Keras MobileNet V2 model and compile the model with the optimizer, loss, and metrics to train with:

In [0]:
keras_mobilenet_v2 = tf.keras.applications.MobileNetV2(
    input_shape=(160, 160, 3), include_top=False)

estimator_model = tf.keras.Sequential([
    keras_mobilenet_v2,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='softmax')
])

# Compile the model
estimator_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

Create an `Estimator` from the compiled Keras model. The initial model state of the Keras model is preserved in the created `Estimator`:

In [0]:
est_mobilenet_v2 = tf.keras.estimator.model_to_estimator(keras_model=estimator_model)

Treat the derived `Estimator` as you would with any other `Estimator`.

In [0]:
IMG_SIZE = 160  # All images will be resized to 160x160

def preprocess(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

In [0]:
def train_input_fn(batch_size):
  data = tfds.load('cats_vs_dogs', as_supervised=True)
  train_data = data['train']
  train_data = train_data.map(preprocess).shuffle(500).batch(batch_size)
  return train_data

To train, call Estimator's train function:

In [0]:
est_mobilenet_v2.train(input_fn=lambda: train_input_fn(32), steps=500)

Similarly, to evaluate, call the Estimator's evaluate function:

In [0]:
est_mobilenet_v2.evaluate(input_fn=lambda: train_input_fn(32), steps=10)

For more details, please refer to the documentation for `tf.keras.estimator.model_to_estimator`.